In [ ]:
experiment = {
            "dataset": {
                "video_path": r"H:\My Drive\Megatron_DeepFake\dataset",
                "num_frames": 5,
                "random_initial_frame": False,
                "depth_anything_size": "Small",
                "num_video": 20,
                "frame_threshold": 10,
            },
            "dataloader": {
                "batch_size": 2,
                "repvit_model": "repvit_m0_9.dist_300e_in1k",
            },
            "transformer": {
                "d_model": 384,
                "n_heads": 2,
                "n_layers": 1,
                "d_ff": 1024,
            },
            "train": {
                "learning_rate": 0.001,
                "epochs": 10,
                "log_dir": "../data/runs/exp1",
                "early_stop_counter": 10,
                "train_size": 0.5,
                "val_size": 0.3,
                "test_size": 0.2,
                "seed": 42,
            },
        }

In [ ]:
from megatron.trainer import Config,Trainer

config = Config(**experiment)
trainer = Trainer(config)
trainer.train()


In [ ]:
%load_ext tensorboard

# Start TensorBoard
log_dir = config.train.log_dir
%tensorboard --logdir {log_dir}